In [1]:
from riemann import tx as rtx
from riemann import utils
from datetime import datetime, timezone
import pandas as pd

import bitcoin
from bitcoin.rpc import Proxy
from bitcoin.core import b2lx, lx
import binascii

bitcoin.SelectParams('mainnet')
bitcoind = Proxy()

# from sqlalchemy import create_engine
# from sqlalchemy import Column, Integer, Text, MetaData, Table, String

# engine = create_engine('sqlite://')
# df = pd.DataFrame(columns=['txid', 'block_height', 'input_num', 'output_num', 'output_val', 'witness_idx', 'witness_data'])
# witness_data = Table(
#     Column('txid', String)
#     Column
# )

# Can specify m-of-n to get specific multisig type
# Use bytestrings for m-of-n b'\x52'
def get_multisig_witnesses(blocks, m=None, n=None):
    witness_data = []
    for block in blocks:
        block_height = block.height
        block_unix_time = block.header.timestamp.replace(tzinfo=timezone.utc).timestamp()

        for tx in block.transactions:
            if not tx.is_segwit:
                continue

            try:
                dtx = rtx.Tx.from_bytes(tx.hex)     
            except:
                print(f'Transaction errored. Txid: {tx.txid} Hex: {tx.hex}')   
                continue

            # LN multisigs have Segwit version 00 at the start of the stack
            if dtx and len(dtx.tx_witnesses[0].stack) > 0 and (dtx.tx_witnesses[0].stack[0] != b'\x00'):
                continue    

            txid = tx.txid
            input_num = tx.n_inputs
            output_num = tx.n_outputs

            # FIX: This is the total value of the transaction but should be the value of each individual output
            output_val = sum(utils.le2i(out.value) for out in dtx.tx_outs)

            for witness_idx, witness in enumerate(dtx.tx_witnesses):
                if len(witness.stack) == 0:
                    continue

                witness_script = witness.stack[-1].to_bytes()

                # Make sure script ends with op_checkmultisig                
                if witness_script[-1:] != b'\xAE':
                    continue 

                multisig_m = witness_script[1:2]
                if m and multisig_m != m:
                    continue
                    
                multisig_n = witness_script[-2:-1]
                if n and multisig_n != n:
                    continue
                
                # FIX: Avoid double counting when later analyzing data
                # Will be corrected once I modify to lookup input outpoints but that will be very slow
                if witness_idx > 0:
                    output_val = 0
#                 input_val = dtx.tx_ins[witness_idx].outpoint
#                 wd = ''.join(item for item in witness.stack)

                try:
                    outpoint = dtx.tx_ins[witness_idx].outpoint
    #                 print(binascii.hexlify(outpoint.tx_id))
                    outpoint_tx = bitcoind.getrawtransaction(outpoint.tx_id)
    #                 print(utils.le2i(outpoint.index))
                    outpoint_val = outpoint_tx.vout[utils.le2i(outpoint.index)].nValue
#                     print(f'Got outpoint value for: txid: {binascii.hexlify(utils.change_endianness(outpoint.tx_id))} index: {binascii.hexlify(utils.change_endianness(outpoint.index))}')
                except:
#                     print(f'Unable to get value for outpoint. txid: {binascii.hexlify(utils.change_endianness(outpoint.tx_id))} index: {binascii.hexlify(utils.change_endianness(outpoint.index))}')
                    outpoint_val = 'Missing'
            
                witness_data.append({'block_unix_time' : block_unix_time,
                                   'txid' : txid,
                                   'block_height' : block_height,
                                   'input_num' : input_num,
                                   'outpoint_val' : outpoint_val,
                                   'outpoint_txid' : binascii.hexlify(utils.change_endianness(outpoint.tx_id)),
                                   'outpoint_index' : utils.le2i(outpoint.index),
                                   'output_num' : output_num,
                                   'output_val' : output_val,
                                   'witness_idx' : witness_idx 
#                                    'witness_data' : wd
                                    })
#                 outpoint_txid = dtx.tx_ins[witness_idx].outpoint.tx_id
#                 input_val = 
#                 for data in witness.stack

    return witness_data

In [ ]:
# For making csvs
import os
import csv
from blockchain_parser.blockchain import Blockchain

blockchain = Blockchain(os.path.expanduser('/mnt/volume_nyc1_03/blocks/'))

# start = 507000
# end = 600000
start = 507000
end = 600000
with open(f'data-sets/multisig_witnesses_{start}_{end}.csv', 'w', buffering=1, newline='') as csvfile:
    fieldnames = ['block_unix_time', 'txid', 'block_height', 'input_num', 'outpoint_val', 'outpoint_txid', 'outpoint_index', 'output_num', 'output_val', 'witness_idx']
    writer = csv.DictWriter(csvfile, fieldnames=fieldnames)
    writer.writeheader()
    
    step = 100
    for i in range(start, end, step):
        print(f'Getting blocks {i} to {i+step-1}')
        blocks = blockchain.get_ordered_blocks(os.path.expanduser('/mnt/volume_nyc1_03/data/blocks/index'), start=i, end=i+step-1)
        witness_data = get_multisig_witnesses(blocks, m=b'\x52', n=b'\x52')
        
        print(f'Writing blocks {i} to {i+step-1}')
        for data in witness_data:
            writer.writerow(data)

Getting blocks 507000 to 507099
Writing blocks 507000 to 507099
Getting blocks 507100 to 507199
Writing blocks 507100 to 507199
Getting blocks 507200 to 507299
Writing blocks 507200 to 507299
Getting blocks 507300 to 507399
Writing blocks 507300 to 507399
Getting blocks 507400 to 507499
Writing blocks 507400 to 507499
Getting blocks 507500 to 507599
Writing blocks 507500 to 507599
Getting blocks 507600 to 507699
Writing blocks 507600 to 507699
Getting blocks 507700 to 507799
Writing blocks 507700 to 507799
Getting blocks 507800 to 507899
Writing blocks 507800 to 507899
Getting blocks 507900 to 507999
Writing blocks 507900 to 507999
Getting blocks 508000 to 508099
Writing blocks 508000 to 508099
Getting blocks 508100 to 508199
Writing blocks 508100 to 508199
Getting blocks 508200 to 508299
Writing blocks 508200 to 508299
Getting blocks 508300 to 508399
Writing blocks 508300 to 508399
Getting blocks 508400 to 508499
Writing blocks 508400 to 508499
Getting blocks 508500 to 508599
Writing 

Writing blocks 519800 to 519899
Getting blocks 519900 to 519999
Writing blocks 519900 to 519999
Getting blocks 520000 to 520099
Writing blocks 520000 to 520099
Getting blocks 520100 to 520199
Writing blocks 520100 to 520199
Getting blocks 520200 to 520299
Writing blocks 520200 to 520299
Getting blocks 520300 to 520399
Writing blocks 520300 to 520399
Getting blocks 520400 to 520499
Writing blocks 520400 to 520499
Getting blocks 520500 to 520599
Writing blocks 520500 to 520599
Getting blocks 520600 to 520699
Writing blocks 520600 to 520699
Getting blocks 520700 to 520799
Writing blocks 520700 to 520799
Getting blocks 520800 to 520899
Writing blocks 520800 to 520899
Getting blocks 520900 to 520999
Writing blocks 520900 to 520999
Getting blocks 521000 to 521099
Writing blocks 521000 to 521099
Getting blocks 521100 to 521199
Writing blocks 521100 to 521199
Getting blocks 521200 to 521299
Writing blocks 521200 to 521299
Getting blocks 521300 to 521399
Writing blocks 521300 to 521399
Getting 

In [ ]:
# For testing
import os
from blockchain_parser.blockchain import Blockchain

blockchain = Blockchain(os.path.expanduser('/mnt/volume_nyc1_03/blocks/'))
start = 100000
end = 100001

blocks = blockchain.get_ordered_blocks(os.path.expanduser('/mnt/volume_nyc1_03/data/blocks/index'), start=start, end=end, cache=f'index-cache_{start}_{end}.pickle')
result = get_multisig_witnesses(blocks, m=b'\x52', n=b'\x52')

display(result)

In [ ]:
import bitcoin
from bitcoin.rpc import Proxy
from bitcoin.core import b2lx, lx
bitcoin.SelectParams('mainnet')

bitcoind = Proxy()
# print(b2lx(bitcoind.getbestblockhash()))
raw_tx = bitcoind.getrawtransaction(lx('6359f0868171b1d194cbee1af2f16ea598ae8fad666d9b012c8ed2b79a236ec4'))
print(raw_tx.vout[1].nValue)

In [ ]:
# import json
import requests

tx = b'c82a53b3f10c4e72b81ecc89cf1532187ea280790cd5aa1b5bdfdec921147a7c'
postdata = {'version': '1.1',
                       'method': 'getrawtransaction',
                       'params': tx
                      }

headers = {
    'Host': 'localhost',
    'User-Agent': 'AuthServiceProxy/0.1',
    'Content-type': 'application/json',
}

response = requests.post('http://localhost:8332', data=postdata, headers=headers)
# json_resp = response.json()

print(response)

In [10]:
import bitcoin
from bitcoin.rpc import Proxy
from bitcoin.core import b2lx, lx, x
import binascii

bitcoin.SelectParams('mainnet')
bitcoind = Proxy()

tx = 'b9e3b3366b31136bd262c64ff6e4c29918a457840c5f53cbeeeefa41ca3b7005'

print(tx)

outpoint_tx = bitcoind.getrawtransaction(tx)
print(outpoint_tx)

b9e3b3366b31136bd262c64ff6e4c29918a457840c5f53cbeeeefa41ca3b7005


TypeError: a bytes-like object is required, not 'str'